In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device('cuda')

In [3]:
device  # Checking weather GPU or CPU is being used

device(type='cuda')

In [4]:
torch.cuda.mem_get_info()

(5368709120, 6442123264)

In [5]:
torch.cuda.empty_cache() 

In [6]:
torch.cuda.mem_get_info()

(5368709120, 6442123264)

In [7]:
# Hyper-parameters
num_epochs = 10
batch_size = 4
learning_rate = 0.001

In [8]:
transform = transforms.Compose(
                [transforms.ToTensor(),
                 transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])   # Normalizing the data

In [9]:
train_dataset = torchvision.datasets.CIFAR10(root = './data', train =True, download=True, transform = transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size= batch_size, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root = './data', train =False, download=True, transform = transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size= batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [11]:
#CNN
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
model = ConvNet().to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [13]:
torch.cuda.mem_get_info()

(5366611968, 6442123264)

In [14]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [2000/12500], Loss: 2.2859
Epoch [1/10], Step [4000/12500], Loss: 2.2997
Epoch [1/10], Step [6000/12500], Loss: 2.2872
Epoch [1/10], Step [8000/12500], Loss: 2.2317
Epoch [1/10], Step [10000/12500], Loss: 2.2537
Epoch [1/10], Step [12000/12500], Loss: 2.2261
Epoch [2/10], Step [2000/12500], Loss: 2.0235
Epoch [2/10], Step [4000/12500], Loss: 1.7604
Epoch [2/10], Step [6000/12500], Loss: 2.0179
Epoch [2/10], Step [8000/12500], Loss: 1.7326
Epoch [2/10], Step [10000/12500], Loss: 1.6597
Epoch [2/10], Step [12000/12500], Loss: 2.7043
Epoch [3/10], Step [2000/12500], Loss: 1.1232
Epoch [3/10], Step [4000/12500], Loss: 2.3969
Epoch [3/10], Step [6000/12500], Loss: 1.3320
Epoch [3/10], Step [8000/12500], Loss: 2.2581
Epoch [3/10], Step [10000/12500], Loss: 1.3527
Epoch [3/10], Step [12000/12500], Loss: 1.3175
Epoch [4/10], Step [2000/12500], Loss: 1.1168
Epoch [4/10], Step [4000/12500], Loss: 0.8568
Epoch [4/10], Step [6000/12500], Loss: 1.0584
Epoch [4/10], Step [8000/125

In [15]:
torch.cuda.mem_get_info()

(5295308800, 6442123264)

In [16]:
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Finished Training


In [17]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 56.22 %
Accuracy of plane: 55.7 %
Accuracy of car: 59.9 %
Accuracy of bird: 47.2 %
Accuracy of cat: 24.9 %
Accuracy of deer: 36.0 %
Accuracy of dog: 47.8 %
Accuracy of frog: 74.6 %
Accuracy of horse: 66.2 %
Accuracy of ship: 84.5 %
Accuracy of truck: 65.4 %


In [18]:
torch.cuda.mem_get_info()

(5295308800, 6442123264)